Asignatura: Tipología y ciclo de vida de los datos.
Alumnos: PABLO CHILLERÓN BEVIÁ y EVGENY MUZAREV GEVORGIAN

Práctica 1.

Esta practica consiste de 2 fases: Fase 1 Scraping y Fase 2 El Análisis. 



# FASE 1 SRAPING.

## Descripción.

Con este script lo que se pretende es, recolectar las ofertas del portal bookings.com de las próximas dos semanas(14 días), a partir del momento de ejecución del script. Los criterios de las ofertas son: país España, actividad Playa, 1 persona adulta sin niños, para una noche. Es decir, el portal bookings.com nos muestra todas las ofertas de hoteles en España, que están situados en zonas costeras, cerca de la playa, para pernoctar una persona sin niños.

La forma en la que se preve recoger y presentar los datos de las ofertas es un dataframe, que contendra las siguientes columnas(variables): ['name','stars','addr','country','city','postal_c','score','comments','beach','service_1','service_2',...'service_n', 'price']

Las ofertas se presentan en forma de bloques, unos 25 bloques por página, que contienen el precio de la oferta y una información breve. Estos bloques contienen urls que llevan a las fichas de ofertas con la información más detallada.

Este script recoge el nombre del hotel de la oferta (la variable 'name'), el nº de estrellas (la variable 'stars'), la dirección completa (la variable 'addr'), el país (la variable 'country'), la ciudad (la variable 'city'), el código postal (la variable 'postal_c'), el rating del hotel entre los usuarios (la variable 'score'), el nº de comentarios sobre hotel (la variable 'comments'), el rating sobre la playa (la variable 'beach'), los servicios que ofrece el hotel, como wifi gratis, parking, piscina, etc. (las variables llevarán el nombre de los servicios), estos servicios se presentarán en forma binaria: 1 el servicio presente en la oferta, 0 no. Finalmente, se recoge el precio de la oferta (la variable 'price'). 

Se prevé que el script recoja un dataframe de unos aproximadamente 15-16 variables, por unos 14000 registros (ofertas), ya que el portal bookings.com muestra unas mil ofertas por fecha.

La lógica del script es la siguiente. Primero, se recogen las urls de todas las ofertas por fechas. Segundo, se extraen todas las variables de cada una de las urls recogidas en el primer paso.

Para esto, se divide el proceso en dos. El primer proceso recolecta las urls de las ofertas, que son aproximadamente 14000 urls, y dura unos 30 minutos. El segundo proceso, que se encarga de la extracción de los datos, puede procesar unas 1000 páginas por hora, si se utiliza de manera secuencial, es decir una url detrás de otra, y unas 1000 urls cada 5 minutos, si se utiliza multi-threading. 

Si no se utiliza multi-threading, se tardaría más de 10 horas en completar el proceso. Utilizando multi-threading, se tarda 2-3 horas, aun así, el proceso completo es <strong>MUY LENTO</strong>.



Para probar la funcionalidad del script, el nº de días está establecido a sólo 1 día, en lugar de 14. Mientras que el dataset utilizado para el análisis, contiene 14 días. Es decir, al ejecutar todo el código, se generará un dataset pequeño de aproximadamente 1000 entradas(depende de la cantidad de ofertas que haya el día que se ejecuta el código) después de completarse la fase de scraping(dataframe.csv), y para la fase del análisis, se utiliza el dataset completo de 14 días, que contiene 13943 registros(dataframe_13943.csv), generado con anterioridad. Todo esto hace que el código tarda unos 7-10 minutos como mucho en ejecutarse completamente. Con lo cual, se puede ejecutarlo sin miedo.

## 1. Extracción de las urls de las ofertas.


La extracción consiste en crear un diccionario, que contendrá todas las urls de todas las ofertas, para cada una de las fechas, empezando desde la fecha en la cual se ejecuta el script + 13 días. El diccionario tendrá la fecha para la que se hace la búsqueda como la clave (key), y una lista de urls para esta fecha como valor(value).

Para esto, creamos una lista de las urls de las páginas de respuesta a la búsqueda para cada una de las fechas. Estas páginas aparte de presentar los bloques con ofertas, también abajo del todo, presentan botones-enlaces a las siguientes páginas, que muestran otros 25 resultados(1,2,3,...n,..9). Las urls de estas páginas son distintas a las urls de las primeras páginas. Con lo cual, es necesario generar un segundo tipo de las urls a partir de la primera, modificándolos un poco y añadiendo la parte de búsqueda que siempre acaba en el número de resultados a mostrar (=25, =50, =75, etc). Abajo de la página, se muestran botones para pasar a la siguiente página (1,2,3....8,9). Sabiendo qué número es el último, sabemos cuantos pasos de 25 hay que dar. Por ejemplo, si el último número de los botones acaba en 7, sabemos que hay que incrementar 7 veces en 25 el último número de la url: =25 (se muestran resultados de 26 a 50), =50 (se muestran resultados de 51 a 75), =75(de 76-100),..., y así hasta 7 veces, 7x25=175. 

De esta manera, pasando por cada una de estas urls que contienen 25 ofertas, extraemos las urls de fichas de estas 25 ofertas. Es decir, primero construimos unas urls, para poder extraer otras.

Estas últimas, se utilizarán ya para extraer los datos.

Aparte de las urls, también se extraen los precios de las ofertas que se muestran en los bloques, ya que no ha sido posible extraerlos en el segundo proceso, por la complejidad de la implementación. Esto es debido a que los parsers que se utilizan en la librería bs4 (html_parser y lxml) son muy caprichosos y procesan html de las páginas de manera distinta. Al igual que pasa con los módulos de la librería requests, depende de cual se utiliza para establecer la conexión, influye, por lo visto, en cómo manda el código html el servidor. Todo esto presenta ciertas dificultades a la hora de implementar el parsing, ya que, primero el código html que el servidor manda al nuestro navegador y el que vemos en el 'Element Inspector' puede no coincidir con el código html que manda el servidor a petición del módulo de la librería 'requests'. Luego, cada parser (html_parser y lxml) procesa el código de manera distinta, hasta el punto que los tags que vemos en el 'Element Inspector', no están presentes en html que tenemos cargado en bs4. Realmente, ha sido bastante complicado ir jugando con los parsers y los módulos de 'request', para encontrar la combinación óptima.

Por ejemplo, para extraer las urls de las fichas de ofertas y los nombres de hoteles, es mejor utilizar css selector, 'html_parser' y requests.get(), mientras que para extraer los datos de las ofertas, es mejor utilizar
'lxml' parser y urllib.request.urlopen().


In [ ]:
# Cargamos librerías necesarias

import requests
from bs4 import BeautifulSoup
from datetime import timedelta
import time
import datetime as datetime
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import urllib.request
import lxml
import pickle
headers = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5"\
           "37.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
import warnings
warnings.filterwarnings('ignore')


### La función url_l_f(), que compone la urls de las páginas de resultados.

Esta función es la que compone las urls de las páginas de los resultados de las ofertas, y que pasa estas urls a la siguiente función baige(), encargada de extraer las urls de las fichas de las ofertas y los precios.

In [ ]:
def url_l_f(url,pags):
    
    ##### Esta función recibe de 2 argumentos: la url inicial de los resultados    #####
    ##### con las fichas de los hoteles, que servirá de base a partir de la cual   #####
    ##### se van a construir las siguientes url'es consecutivas de la muestra de   #####
    ##### los resultados(25,50,75,100,125,...,pags) y el número de páginas(pags)   #####
    ##### de 25 resultados cada una, hasta que hay que contruir las URLs           ##### 
    ##### Finalmente, la función pasa cada una de las url'es generadas a la        #####
    ##### baige(), que se encarga de extraer cada una de las url de las fichas de  #####
    ##### hoteles, y sus precio y el nombre.                                       #####
    
    # Iniciamos el número de resultados a mostrar con un 0
    b = 0
    # Para cada una de las páginas
    for i in range(0,pags):
        # Incrementamos un paso(cada paso=página de 25 resultados)
        b = b+25
        # Añadimos el paso a la url base y la pasamos a la función baige()
        baige(url+str(b),0)
        


### La función baige(), que extrae las urls de las fichas de ofertas y precios.

Esta función recibe las urls procedentes de la función anterior url_l_f(), una a una, realiza una conexión 'request' al bookikgs.com y extrae las urls de las fichas de las ofertas y sus precios. Las urls, cada una con su identificador, se agregan a un diccionario directamente, igual que los precios y los nombres, cada uno con su identificador, se agregan en dos diccionarios separados. Los identificadores se utilizan para poder recrear el orden original de los registros, debido a que el orden se altera al utilizar multi-threading, y el orden original lo necesitamos para unir correctamente los precios al dataframe.

El motivo por el cual se extrae el precio en esta parte del proceso y no junto con el resto de los datos, ya se ha expuesto mas arriba, y es que extraer el precio desde los bloques de ofertas de la página de resultados es más fácil que desde las fichas, que son casi imposibles de acceder.



In [ ]:
def baige(url, x):
    
    ##### Esta función recibe una url de la página que contiene las fichas de      #####
    ##### hoteles que salen en los resultados de la búsqueda, realiza una llamada  #####
    ##### al servidor con la url pasada utilizando la librería requests, crea un   #####
    ##### objeto 's' de bs4 con la sesión y el parser, scrapea las url'es de cada  #####
    ##### una de las fichas de los hoteles que salen en la página y los agrega a   #####
    ##### la lista ls[]. También devuelve las el objeto 's' con la sesión, y       #####
    ##### también imprime por pantalla las urls que va agregando y el contador.    #####
    
    ##### Otra funcionalidad que ha sido añadida posteriormente, es recoger los    #####
    ##### precios y nombres de los hoteles. 
    
    # La llamada al servidor y el objeto 's'
    r = requests.get(url, headers=headers)
    s = BeautifulSoup(r.content, 'lxml')
    
    # El contador
    global d
    # Los identificadores
    global id_u # de url
    global id_p # de precio
    global id_d # de fecha
    
    
    # Accedemos a los elementos de las fichas que contienen partes de url y
    # recreamos url completa
    for f in s.select(".f0b5ba03dc"):
        
        # Imprimimos url por pantalla
        print(f.a['href'].strip())
        
        # Imprimimos e incrementamos el contador
        print(d)
        d=d+1
        
        # Agregamos url al diccionario con su correspondiente id_u
        dic_u[id_u]= f.a['href'].strip()
        # Incrementamos identificador de url
        id_u=id_u+1
        # Agregamos la fecha al diccionario con su correspondiente id_d
        dic_d[id_d] = date
        # Incrementamos id_d 
        id_d=id_d+1
    # Accedemos al precio, lo imprimimos por pantalla y lo agregamos 
    # al diccionario con su correspondiente id_p
    for t in s.select('._e885fdc12'):
        print(t.text.split('€')[1].strip())
        dic_p[id_p] = t.text.split('€')[1].strip()
        id_p=id_p+1
    
    # Por si acaso, controlamos sincronización
    if not((id_u==id_p)&(id_p==id_d)):
        # Si pasa algo, agregamos el nº de id en el cual ha pasado algo
        alarm.append(id_u)
        alarm.append(id_n)
        alarm.append(id_p)
        alarm.append(id_d)
    # Devolvemos la sesión
    if x==1:
        return(s)
    else:
        print("ok")


### El script que arranca la primera parte del proceso, la extracción de urls.

Primero se crea una lista de fechas, luego se componen las correspondientes urls para esta fechas y se van pasando, una a una a la función baige(), la cual descarga el contenido de la primera página de los resultados de la búsqueda, extrae la url y el precio del hotel, y los guarda en los diccionarios.

También devuelve la sesión 's' de bs4 con la primera página cargada, para calcular el número de las páginas de resultados.

A continuación, se llama a la función url_l_f(), pasándole las url de las siguientes páginas de resultados, a las cuales se les añade 25,50,75,100,..., hasta el número de las páginas, que se pasan como el segundo argumento. Esta función pasa las urls obtenidas a la función baige(), que extrae las urls de las fichas y los precios.

Finalmente, se agregan las urls y precios extraídos a sus respectivos diccionarios.

In [ ]:
# Busca los próximos días como fechas
prox_dias = 1 # Está puesto a 1 día, para que la ejecución no tarde demasiado, 
              # aunque para generar el dataset para el análisis, se ha utilizado los 14 días
dates = [datetime.date.today() + timedelta(days=j) for j in range(0,prox_dias)]

# Se crea una lista de url'es, por si acaso
urls=[]
# Se crea una especie de controlador para controlar la sincronización de
# extracción de los datos
alarm = []
# Se crea un contador
d=1
# Se crean identificadores para la extracción de urls, precios y fechas
id_u=0
id_p=0
id_d=0

# Creamos el diccionario para las urls{id_u:url} 
dic_u = {}
# Creamos diccionario de precios{id_p:precio}
dic_p = {}
# Creamos el diccionario para fechas{id_p:date}
dic_d = {}

# Para cada una de las fechas:
for date in dates:
    # Creamos una lista en la que se van a agregar los valores
    ls = []
    
    # Extraemos componentes de las fechas de entrada/salida:      
    checkin_date = date  
    checkout_date = date + timedelta(days=1) 
    checkin_day = checkin_date.day
    checkin_month = checkin_date.month
    checkin_year = checkin_date.year
    checkout_day = checkout_date.day
    checkout_month = checkout_date.month
    checkout_year = checkout_date.year
    
    # Recreamos url de la primera página de resultados de la búsqueda  
    url_2 = 'https://www.booking.com/searchresults.es.html?aid=309654&label='\
    'booking-be-es-4hlnAGESyHVwkqBgv3aKIwS479808260559%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg'\
    '%3Afi%3Atikwd-13273066%3Alp1005413%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YcsZ-Id2vkzIfTm'\
    'YhvC5HOg&sid=c4b3f6f420352b7e6268dbe166abff7d&tmpl=searchresults&ac_click_type=b&ac_position=0&'\
    'checkin_month='+str(checkin_date.month)+'&checkin_monthday='+str(checkin_date.day)+'&'\
    'checkin_year='+str(checkin_date.year)+'&checkout_month='+str(checkout_date.month)+'&'\
    'checkout_monthday='+str(checkout_date.day)+'&checkout_year='+str(checkout_date.year)+'&'\
    'city=-373226&class_interval=1&dest_id=197&dest_type=country&from_sf=1&g'\
    'roup_adults=1&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A&'\
    'sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&src=searchresults&srpvid='\
    '2cc557b3d2e20074&ss=Espa%C3%B1a&ss_raw=Esp&ssb=empty&ssne=Benidorm&ssne_untouched=Benidorm&top_ufis=1&'\
    'sig=v1RzUHRY77&nflt=popular_activities%3D302%3Bht_id%3D204%3B&percent_htype_hotel=1&rsf='
    
    # Recreamos el segundo tipo de url de las páginas de resultados(las que muestran 
    #1-25, 26-50,51-75,...,etc), primera parte
    a = 'https://www.booking.com/searchresults.es.html?aid=309654&label='\
    'booking-be-es-4hlnAGESyHVwkqBgv3aKIwS479808260559%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg'\
    '%3Afi%3Atikwd-13273066%3Alp1005413%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YcsZ-Id2vkzIfTm'\
    'YhvC5HOg&sid=c4b3f6f420352b7e6268dbe166abff7d&tmpl=searchresults&ac_click_type=b&ac_position=0&'\
    'checkin_month='+str(checkin_date.month)+'&checkin_monthday='+str(checkin_date.day)+'&'\
    'checkin_year='+str(checkin_date.year)+'&checkout_month='+str(checkout_date.month)+'&'\
    'checkout_monthday='+str(checkout_date.day)+'&'\
    'checkout_year='+str(checkout_date.year)+'&city=-373226&class_interval=1&dest_id=197&dest_type=country&'
    
    # Segunda parte
    b = 'dtdisc=0&from_sf=1&group_adults=1&group_children=0&inac=0&index_postcard=0&label_click=undef&'\
    'nflt=popular_activities%3D302%3Bht_id%3D204%3B&no_rooms=1&percent_htype_hotel=1&postcard=0&'\
    'raw_dest_type=country&room1=A&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&'\
    'src=searchresults&srpvid=65e56cb746a10078&ss=Espa%C3%B1a&ss_all=0&ss_raw=Esp&ssb=empty&sshis=0&'\
    'ssne=Benidorm&ssne_untouched=Benidorm&top_ufis=1&sig=v1hAlQhinU&rows=25&offset='
    
    # Juntamos las dos partes y tenemos una urla inicial, a la cual vamos a ir
    # añadiendo los resultados(25,50,75,100,...) que se mostrarán en la página.
    # 25 resultados por página.
    p_url = a+b
    
    #urls.append(url_2)
    
    # Añadimos el primer valor a la lista, que será la fecha
    ls.append(date)
    

    
    # Llamamos la función baige() que extrae url de las fichas de los hoteles que se muestran en
    # la página de resultados y los guarda en una lista. La función también devuelve s(un objeto 
    # de bs4 con la primera página de resultados de la búsqueda para cada fecha)
    s=baige(url_2,1)
    
    ### Cálculo de nº de páginas de resultados: Nºres.devueltos/25 = nºde páginas ###
    
    # Accedemos al elemento de la página, que muestra el número de resultados de la búsqueda y 
    # si devuelve más de 1000 resultados, por ejemplo 1346, lo dejamos a 1000, ya que el servidor
    # de booking no muestra más de 1000 resultados, que son 40 páginas de 25 resultados en cada una
    if '.' in s.find('div',{'id':'bodyconstraint'}).div.find('div',{'id':'searchresultsTmpl'}).div.div.h1.text.split(' ')[1].strip():
        pags = 39
    # Si la búsqueda devuelve menos de 1000 resultados, se deviden entre 25, se redondea
    # al número mayor siguiente, si la división devuelve decimales
    else:
        v = int(s.find('div',{'id':'bodyconstraint'}).div.find('div',{'id':'searchresultsTmpl'}).div.div.h1.text.split(' ')[1].strip())
        pags = int(v/25) + (v%25 > 0)
    
    
    # Se llama la función url_l_f(), a la que se le pasa la p_url, url que se ha preparado
    # anteriormente, que es la url inicial de todas las páginas de resultados, que acaba en
    # "https://www.booking.com/../..offset=", y a la que se le va añadiendo offset=25,
    # offset=50, offset=75, offset=100, etcétera, en función del número de páginas(pags).
    # Luego, se guarda la url de cada una de las fichas de hoteles que se muestran en
    # la página, unas 25, y se van añadiendo a la lista ls[].
    url_l_f(p_url,pags)
    
    # Finalmente, guardamos la lista ls[] en el diccionario que hemos creado anteriormente.
    # Para cada fecha(que son 14), guardaremos la lista ls[], con sus correspondientes 
    # url'es, que hemos recogido con las funciones baige()(para recoger url's de primera
    # página) y url_l_f(), para recoger url'es del resto de la página, para cada una de
    # las 14 fechas. 
    #dic[date]=ls

        

In [ ]:
# Vemos si en la lista alarm hay algún valor
if bool(len(alarm)):
    print(alarm)
else:
    print(id_u)
    print(id_p)
    print(id_d)

In [ ]:

# Guardamos el diccionario con las urls
a_file = open("dic_u.pkl", "wb")

pickle.dump(dic_u, a_file)

a_file.close()

# Guardamos el diccionaro de precios
a_file = open("dic_p.pkl", "wb")

pickle.dump(dic_p, a_file)

a_file.close()


# Guardamos el diccionaro de fechas
a_file = open("dic_d.pkl", "wb")

pickle.dump(dic_d, a_file)

a_file.close()

## 2. Extracción de los datos de las ofertas.

Esta parte de proceso se encarga de escrapear cada una de las urls obtenidas en la fase anterior. Al final de esta parte, tendremos una lista enorme con los valores de las variables, que luego se pasará al formato del dataframe, y un diccionario con los servicios que ofrece cada hotel y que se unirá posteriormente al dataframe resultante en forma binaria: 1 si el servicio está presente en la oferta, 0 si no.

Al implementar multi-threading, se altera el orden en el cual se agregan los valores a la lista de las urls pasadas. Es decir, el orden de los valores en la lista devuelta no es el mismo en el que se pasaban las urls. Pero en nuestro caso el orden importa, ya que todavía tenemos que unir los precios al dataframe, que están en el mismo orden en el que se han extraído. Con lo cual, es necesario inventar un identificador que indique el orden en el cual se pasan las urls de entrada, para poder luego reconstruir el orden original de la salida.

In [ ]:
# Creamos la lista resultante que posteriormente se pasará al
# dataframe
res=[]
# Creamos el diccionario con nombres de hoteles y servvicios
# que ofrecen
dic={}
# Creamos contador
d_count = 0

### La función dop(), que se encarga de extraer los datos.

Esta función recibe una url, fecha y un identificador coomo argumentos, realiza una conexión 'request' para la url de la ficha de oferta pasada y va extrayendo los datos, añadiéndolos a una lista temporal, como si fuera una fila del dataframe, que posteriormente se incorporará a la lista resultante res[]. Esta fila tendrá la siguiente estructura: ['id','date','name','stars','addr','country','city','postal_c','score','comments','beach'].

El diccionario de servicios tendrá i_d como la clave y una lista de servicios en forma ['nombre','servicio_1', servicio_2, servicio_3,..., servicio_n] como valor. Posteriormente se unirán los servicios que presta cada hotel en su oferta, al dataframe resultante en forma binaria: 1 si está presente, 0 si no.

El identificador i_d se utilizará para reconstruir el orden original de las urls.

In [ ]:
def dop(ur, date, i_d):
    
    ''' Esta función recibe 3 argumentos, una url, una 
        fecha(date) y un identificador(i_d). No devuelve nada.
        Realiza una conexión request con el servido para la
        url pasada de argumento, extrae los datos de ofertas
        de las páginas y los agrega a la lista res[] y al
        diccionario dic[]'''
    
    # Declaramos la variable del contador como global
    global d_count
    
    # Creamos una lista temporal que dura una iteración
    lst = []
    # Agregamos identificador a la lista
    lst.append(i_d)
    # Agregamos la fecha a la lsita, que será el valor de la primera
    # columna "Date" del dataframe
    lst.append(date)
    # Imprimimos la fecha por pantalla
    print(date)
    # Creamos sesión y objeto de bs4 con la url
    r = urllib.request.urlopen(ur)
    soup = BeautifulSoup(r, "lxml")
        
    # Utilizamos la construcción try-except para que no se nos
    # detenga la ejecución en el caso de errores
    try:
       # Accedemos al nombre del hotel
        nombre = soup.find('div',{'id':'bodyconstraint'}).\
            find('div',{'id':'bodyconstraint-inner'}).\
            find('div', {'id':'wrap-hotelpage-top'}).\
            find('div',{'class':'hp__hotel-title'}).h2.text.split('\n')[2].strip()
        # Lo imprimimos por pantalla
        print('Nombre: '+str(nombre))
        # Agregamos el nombre a la lista, que será el valor de la segunda
        # columna del dataframe 
        lst.append(nombre)
            
    # En el caso del error, imprimimos un mensaje por pantalla
    # y simplemente agregamos un nan con el número de iteración a la lista
    except:
        print('nombre nan')
        nombre = str('nan_'+ str(d_count))
        lst.append(nombre)
        
    # Extraemos el número de estrellas
    try: 
        # Accedemos al elemnto que muestra las estrellitas y contamos
        # el número de <span>(que es cada estellita) que contiene
        stars = len(soup.find('div',{'id':'bodyconstraint'}).\
                             find('div',{'id':'bodyconstraint-inner'}).\
                             find('div', {'id':'wrap-hotelpage-top'}).\
                             find('div',{'class':'hp__hotel-title'}).\
                             find('span',{'class':'hp__hotel_ratings'}).span.div.div.\
                             span.div.span.find_all('span'))
            
        # Mostramos el número de estrellas por pantalla
        print('Estrellas: '+str(stars))
        # Agregamos el número de estrellas a la lista, que será el
        # valor de la "Stars" del dataframe
        lst.append(stars)
            
    # En el caso del error, imprimimos un mensaje por pantalla y
    # agregamos un nan a la lista
    except:
        print('estrellas_nan')
        stars = 'nan'
        lst.append(stars)
        
    # Extraemos la dirección
    try:
        # Accedemos al elemento que muetsra la drección y la 
        # extraemos
        addr = soup.find('div',{'id':'bodyconstraint'}).\
                          find('div',{'id':'bodyconstraint-inner'}).\
                          find('div', {'id':'wrap-hotelpage-top'}).p.text.split('\n')[5].strip()
            
        # Imprimimos por pantalla
        print('Dirección: '+str(addr))
        # Agregamos la dirección a la lista, que será el valor de la
        # columna "Address" del dataframe, la dirección completa
        lst.append(addr)
            
    # En el caso de error, imprimimos un mensaje y agregamos un nan 
    # a la lista
    except:
        print('addr nan')
        addr = 'nan'
        lst.append(addr)
            
    # También extraemos el país, la ciudad y el código postal
    # por separado.
        
    # Accedemos y extraemos el país
    try:    
        country = soup.find('div',{'id':'bodyconstraint'}).\
                           find('div',{'id':'bodyconstraint-inner'}).\
                           find('div', {'id':'wrap-hotelpage-top'}).p.text.split('\n')[5].\
                           strip().split(', ')[-1].strip()
        # Imprimimos por pantalla
        print('País: '+str(country))
        # Agregamos el país a la lista, que será el valor de la columna
        # "Country" del dataframe resultante
        lst.append(country)
        
    # En el caso del error, imprimimos un mensaje y agregamos 
    # un nan a la lista
    except:
        print('country nan')
        country = 'nan'
        lst.append(country)
            
    # Accedemos y extraemos a la ciudad
    try:
        city = soup.find('div',{'id':'bodyconstraint'}).\
                    find('div',{'id':'bodyconstraint-inner'}).\
                        find('div', {'id':'wrap-hotelpage-top'}).p.text.split('\n')[5].\
                        strip().split(', ')[-2].split(' ')[-1].strip()
            
        # Imprimimos por pantalla
        print('Ciudad: '+str(city))
        # Agregamos la ciudad a la lista, que será el valor de la columna 
        # "City" del datframe
        lst.append(city)
           
    # En el caso del error, avisamos y agregamos un nan a la lista
    except:
        print('city nan')
        city = 'nan'
        lst.append(city)   
        
    # Accedemos y estraemos el código postal
    try:
        postal_c = soup.find('div',{'id':'bodyconstraint'}).\
                            find('div',{'id':'bodyconstraint-inner'}).\
                            find('div', {'id':'wrap-hotelpage-top'}).p.text.split('\n')[5].\
                            strip().split(', ')[-2].split(' ')[0].strip()
            
        # Imprimimos por pantalla
        print('Cód. postal: '+str(postal_c))
        # Agregamos el código a la lista, que será el valor de la columna
        # "Postal_code" del dataframe
        lst.append(postal_c)
            
    # En el caso del error, avisamos y agregamos un nan a la lista
    except:
        print('postal_c nan')
        postal_c = 'nan'
        lst.append(postal_c)
            
    # Accedemos y extraemos el valor de score
    try:
        score = soup.find('div',{'id':'bodyconstraint'}).\
                          find('div',{'id':'bodyconstraint-inner'}).\
                          find('div',{'id':'hotelTmpl'}).\
                          find('div',{'id':'basiclayout'}).\
                          find('div',{'id':'right'}).\
                          find('div',{'id':'blockdisplay1'}).div.div.div.\
                          find('div',{'class':'hp-gallery-review'}).div.div.a.div.div.div.div.div.text.strip()
        
        # Lo mostramos por pantalla
        print('Score: '+str(score))
        # Agregamos el valore de score a la lista, que será el valor
        # de la columna "Score" del dataframe
        lst.append(score)
            
    # En el caso del error, avisamos y agregamos un nan a la lista 
    except:
        print('score nan')
        score = 'nan'
        lst.append(score)
        
    # Accedemos y extraemos los comentarios
    try:
        # La estructura del elemento que contiene el número de los comentarios
        # puede variar, con lo cual se preven dos tipos. Aunque, debido a la
        # cantidad de páginas, es deficil prever todas las estructuras.
        # Con lo cual, si no es un tipo, se accede al otro.
        if soup.find('div',{'id':'bodyconstraint'}).\
                          find('div',{'id':'bodyconstraint-inner'}).\
                          find('div',{'id':'hotelTmpl'}).\
                          find('div',{'id':'basiclayout'}).\
                          find('div',{'id':'right'}).\
                          find('div',{'class':'hp_nav_bar_wrapper hp-nav-bottom-border bui-spacer--large'}):
            comments = soup.find('div',{'id':'bodyconstraint'}).\
                          find('div',{'id':'bodyconstraint-inner'}).\
                          find('div',{'id':'hotelTmpl'}).\
                          find('div',{'id':'basiclayout'}).\
                          find('div',{'id':'right'}).\
                          find('div',{'class':'hp_nav_bar_wrapper hp-nav-bottom-border bui-spacer--large'}).ul\
                          .span.text.split('(')[1].split(')')[0].strip()
        else:
            comments = soup.find('div',{'id':'bodyconstraint'}).\
                          find('div',{'id':'bodyconstraint-inner'}).\
                          find('div',{'id':'hotelTmpl'}).\
                          find('div',{'id':'basiclayout'}).\
                          find('div',{'id':'right'}).\
                          find('div',{'class':'hp_nav_bar_wrapper hp-nav-bottom-border'}).ul\
                          .span.text.split('(')[1].split(')')[0].strip()


            
        # Imprimimos por pantalla el número de los comentarios
        print('Comments: '+str(comments))
        # Se agregan los comentarios a la lista, que será el valor
        # de la variable "Comments" del dataframe
        lst.append(comments)
        
    # En el caso del error, avisamos y agregamos un nan a la lista
    except:
        print('comments nan')
        comments = 'nan'
        lst.append(comments)

    # Accedemos y extraemos el valor de beach. Igual que con
    # la variable anterior 'comments', se prueban 2 tipos de
    # estructura del elemnto contiene el valor de beach
    try:
            
        if soup.find('div',{'id':'bodyconstraint'}).\
                    find('div',{'id':'bodyconstraint-inner'}).\
                    find('div',{'id':'hotelTmpl'}).\
                    find('div',{'id':'basiclayout'}).\
                    find('div',{'id':'right'}).\
                    find('div',{'id':'blockdisplay1'}).div.\
                    find('div',{'class':'clearfix bh-photo-grid bh-photo-grid--space-down fix-score-hover-opacity'}):
 

                
            beach = soup.find('div',{'id':'bodyconstraint'}).\
                             find('div',{'id':'bodyconstraint-inner'}).\
                             find('div',{'id':'hotelTmpl'}).\
                             find('div',{'id':'basiclayout'}).\
                             find('div',{'id':'right'}).\
                             find('div',{'id':'blockdisplay1'}).div.\
                             find('div',{'class':'clearfix bh-photo-grid bh-photo-grid--space-down fix-score-hover-opacity'}).\
                             find('div',{'class':'hp-gallery-review'}).div.\
                             find('div',{'class':'best-review-score hp_lightbox_score_block'}).span.text.split('\n')[2].strip()
                
        else:
            beach = soup.find('div',{'id':'bodyconstraint'}).\
                             find('div',{'id':'bodyconstraint-inner'}).\
                             find('div',{'id':'hotelTmpl'}).div.div.\
                             find('div',{'id':'wrap-hotelpage-top'}).\
                             find('div',{'class':'hp__hotel-title'}).\
                             find_all('span')[12].text.split('\n')[3].strip()
        
        # Imprimimos por pantalla
        print('Playa: '+str(beach))
        # Agregamos el valor de 'beach' a la lista, que será el
        # valor de la columna "Beach" del dataframe
        lst.append(beach)
            
    # En el caso del error, avisamos y agregamos un nan a la lista
    except:
        print('beach nan')
        beach = 'nan'
        lst.append(beach)


    # Accedemos al elemento de la página que contiene los servicios.
    # Debido a que es un conjunto de servicios, hacemos un diccionario
    # con el nombre del hotel como clave, y la lista de servicios
    # como valor. Luego, agregamos los servicios al dataframe en forma
    # binaria, de manera que cada servicio será una columna del dataframe
    # y contendrá un 1 si el servicio está presente en la fila(oferta), y
    # un 0 si no. 
    try:
        # Accedemos al elemento de la página que contiene el conjunto
        # de servicios
        services = soup.find('div',{'id':'bodyconstraint'}).\
                          find('div',{'id':'bodyconstraint-inner'}).\
                          find('div',{'id':'hotelTmpl'}).\
                          find('div',{'id':'basiclayout'}).\
                          find('div',{'id':'right'}).\
                          find('div',{'id':'blockdisplay1'}).div.\
                          find('div',{'class':'hp_hotel_description_hightlights_wrapper'}).\
                          find('div',{'class':'hotel_description_wrapper_exp hp-description'}).\
                          find('div',{'class':'hp_desc_important_facilities clearfix hp_desc_important_facilities--bui'}).\
                          find_all('div')
        # Creamos una lista
        ls_serv = [] 
           
        # Agregamos la fecha
        ls_serv.append(date)
        
        # Agregamos el nombre
        ls_serv.append(nombre)
        
        # Iteramos por el conjunto de servicios
        for service in services:
            # Extraemos el nombre de servicio y lo agregamos a la
            # lista
            #print(service['data-name-en'].strip())
            ls_serv.append(service['data-name-en'].strip())
        # Agregamos el nombre del hotel con sus correspondientes
        # servicios al diccionario
        dic[i_d]=ls_serv
                     
    # En el caso de no poder extraer los servicios, avisamos y
    # agregamos un nan a la lista
    except:
        print('services nan')
        dic[i_d]='nan'
        
    # Finalmente, obtenemos una lista, que será una fila del dataframe con 
    # la siguiente estructura: ['name','stars','addr','country','city','postal_c','score','comments',beach]
    # Agregamos esta lista(fila) a la lista resultante, que se transformará en 
    # el dataframe.
    res.append(lst)
        
    # Imprimimos el nº del registro
    print()
    print('Registro nº: '+str(d_count))
    print()
    print()
    # Incrementamos el contador
    d_count = d_count+1
    lock.release()

Arrancamos el multithreading con 20 threads.

In [ ]:
import threading

lock = threading.Semaphore(20)
def dop_mt(url_lst):
    
    global i_d
    thread_pool = []

    for i,url in enumerate(url_lst):
        
        thread = threading.Thread(target=dop, args=(url,dic_d[i],i,))
        thread_pool.append(thread)
        thread.start()
        

        
        lock.acquire()

In [ ]:
dop_mt(list(dic_u.values()))

In [ ]:
# Dejamos unos 30 segundos para que acaben todos los threads
time.sleep(30)

Guardamos la lista res[] y el diccionario dic_serv[].

In [ ]:
a_file = open("res.pkl", "wb")

pickle.dump(res, a_file)

a_file.close()



a_file = open("dic_serv.pkl", "wb")

pickle.dump(dic, a_file)

a_file.close()

## 3. Pasamos los datos al dataframe.

In [ ]:
# Cargamos la lista y el diccionario

a_file = open("res.pkl", "rb")

res_2 = pickle.load(a_file)

a_file.close()


a_file = open("dic_serv.pkl", "rb")

dic_serv = pickle.load(a_file)

a_file.close()

In [ ]:
df = pd.DataFrame(res, columns=['Id','Date','Name','Stars','Address','Country','City',\
                                'Postal_Code','Score','Comments','Beach'])
df

Agregamos las columnas de servicios al dataframe.

Primero, necesitamos extraer los valores únicos de servicios desde el diccionario, para dar nombre a columnas.
Para esto, fundimos las listas de servicios que contiene el diccionario en una única lista.

Luego, pasamos esta lista por set(), que nos devuelve los valores únicos.

Y por último, simplemente añadimos columnas al dataframe con los nombres de estos valores únicos.

In [ ]:
# Creamos la lista
ls = []

# Fundimos las listas en una única, dejando fuera los elementos 0 y 1, 
# ya que son fecha y el nombre, y nosotros sólo necesitamos servicios
for keys, values in dic.items():
    ls.extend(dic[keys][2:])
    
# Pasamos la lista por el set y ya tenemos los valores únicos
serv_l = list(set(ls))

# Se ha colado ahí un valor raro y lo eliminamos
#serv_l.remove('n')

# Creamos las columnas de servicios a partir de la lista de
# valores únicos y las dejamos a 0
for serv in serv_l:
    df[serv] = 0

Ahora, tenemos que cuadrar el diccionario con los servicios que ofrecen los hoteles con el dataframe. Para esto, hemos marcado cada fila del dataframe y cada lista del diccionario con un identificador, que en teoría deberían de corresponder.

Usamos una doble condición para estar más seguros, queremos que coincidan id's y que el nombre del hotel del dataframe corresponda con el nombre que está en la lista de servicios, y una vez identificadas las dos cosas, se procede a buscar, servicio por servicio, si el servicio está en la lista o no. Si está, se agrega un 1 a la columna del correspondiente servicio y fila, si no está - un 0.

In [ ]:
# Iteramos por las filas del dataframe
for index, row in df.iterrows():
    # Para la fila de turno, iniciamos iteraciones por el diccionario
    for key, value in dic.items():
        # Comprobamos si coincide el valor de la columna Id del dataframe
        # con la clave del diccionario, que debería de llevar el mismo id
        if (row.Id==key)&(row.Name==dic[key][1]):
            # En el caso positivo, se procede a buscar, servicio por servicio,
            # si está presente en la lista o no
            for col in df.columns[11:]:
                if col in dic[key][2:]:
                    # Si está presente, se agrega un 1 a la celda del
                    # correspondiente sevicio
                    df.iloc[index, df.columns.get_loc(col)]=1
                else:
                    #df.iloc[index][col] = 0
                    # Se agrega un 0 en el caso negativo
                    df.iloc[index, df.columns.get_loc(col)]=0
        else:
            next
df

Por último, vamos a unir los precios al dataframe. 
Se supone que el diccionario de precios y las urls antes de pasarlas al multi-threading estaban en el mismo orden. Con lo cual, se pasó como argumento un identificador i_d con cada url. Ahora, podemos reconstruir el orden original, utilizando sort por la columna 'Id'. Una vez ordenado el dataframe, simplemente procedemos a unir los precios con el dataframe y ya está.  

In [ ]:
# Cargamos el diccionario
a_file = open("dic_p.pkl", "rb")

dic_p = pickle.load(a_file)

a_file.close()

In [ ]:
# Ordenamos el dataframe por la columna 'Id'
df = df.sort_values('Id')

# Unimos los precios
df['Price'] = list(dic_p.values())
df.shape

In [ ]:
# Guardamos el dataframe de 1 día
df.to_csv('dataframe.csv', index=False)

In [ ]:
# Cargamos el dataframe de 14 días
df = pd.read_csv("dataframe_13943.csv")
df.head(2)

Repasamos un poco el dataset, vemos qué es lo que podemos hacer con los nan's

In [ ]:
# Echamos un vistazo a las columnas que tenemos
df.columns

In [ ]:
# Definimos una función que nos muestre los NaNs de cada columna
def nans(df):
    for col in df.columns:
        if (df[col].isna().any()):
            print("The column '"+col+"' has {} NaNs".format(df[col].isna().sum()))

In [ ]:
nans(df)

In [ ]:
# Empezamos, por ejemplo, con la columna 'City', que tiene sólamente 2 valores NaN
df[df.City.isna()]

Podemos ver que las cuatro columnas que tienen 2 NaNs cada una('Address', 'Country', 'City' y 'Postal_Code'), están en estas dos filas y pertenecen a un mismo hotel 'Vincci Selección Estrella del Mar'. Lo que podemos hacer es mirar qué valores tienen estas columnas en otros registros de este hotel y sustituir manualmente los NaNs por estos valores.

In [ ]:
df[df.Name=='Vincci Selección Estrella del Mar'].head(3)

In [ ]:
# Sustituimos los NaNs
df.iloc[13301,[4,5,6,7]] = ['KM 190,5 A7 (Antigua Carretera N340)'\
                            ' Avenida José de Ribera nº 25 ( Urbanización Real de Zaragoza),'\
                            ' 29604 Marbella, España','España','Marbella', 29604]

df.iloc[12880,[4,5,6,7]] = ['KM 190,5 A7 (Antigua Carretera N340)'\
                            ' Avenida José de Ribera nº 25 ( Urbanización Real de Zaragoza),'\
                            ' 29604 Marbella, España','España','Marbella', 29604]

In [ ]:
nans(df)

Después de arreglar los valores NaNs del hotel 'Vincci Selección Estrella del Mar', ya sólo nos quedan 4 columnas con los NaN's. Vamos a seguir con la columna 'Comments', que tiene 5 NaNs.

In [ ]:
df[df.Comments.isna()]

Después de revisar la página del hotel 'Hôte The Corralejo Beach' en booking.com, resulta que tanto los valores 'Score' como 'Comments' están ausentes. Con lo cual, al haber nada más que 5 registros, podemos prescindir de ellos directamente.

In [ ]:
# Eliminamos los registros del hotel 'Hôte The Corralejo Beach'
df = df.drop(df.index[[8860,12916,12922,13933,8911]])

In [ ]:
# Reseteamos los índices
df.reset_index(drop=True, inplace=True)

In [ ]:
# Volvemos a ver los NaNs
nans(df)

Ya sólo nos quedan tres columnas que contienen los valores NaN.

Vamos a seguir con la variable 'Score', que sólo tiene 11 NaNs


In [ ]:
df[df.Score.isna()]

Vemos que los hoteles 'Añoreta Suites', 'Casa Leon' y 'Arena Surf Hotel' no tienen los valores 'Score' ni tampoco 'Stars', en sus correspondientes páginas en booking.com tampoco los tienen. Con lo cual, también vamos a eliminar estos 11 registros.

In [ ]:
# Eliminamos los registros de los hoteles 'Añoreta Suites', 
#'Casa Leon' y 'Arena Surf Hotel'. También reseteamos los índices. 
df = df.drop(df[df.Score.isna()].index)
df.reset_index(drop=True, inplace=True)

In [ ]:
nans(df)

Hemos hecho lo que hemos podido y ya sólo nos quedan dos columnas('Stars' y 'Beach') con muchos valores NaN. Los vamos a dejar como están, y en el caso de necesidad, si vamos a necesitar trabajar con estas variables, ya pensaremos lo que haremos con ellos.


# FASE 2 EL ANÁLISIS.


## El precio.

En este caso práctico, somos los gerentes de un hotel de 3 estrellas ubicado en una zona de costa de la provincia de Alicante. Nuestro objetivo es analizar nuestros precios con respecto a la competencia directa.
El objetivo de este análisis, es conocer si nuestros precios son competitivos y si es necesario modificarlos.

In [ ]:


# Limpiamos la columna "Beach" que será usada para definir la competencia
df['Beach'] = [str(x).replace(',', '.') for x in df['Beach']]
df.loc[df.Beach=='nan', 'Beach'] = np.nan

Para saber quién es nuestra competencia, nos vamos a centrar simplemente en aquellos hoteles de 3 estrellas, que tienen piscina como nosotros, y que están en la provincia de Alicante. Es posible modificar esta definición según la conveniencia de la temporada.

In [ ]:
# Definimos la competencia: hoteles de 3 estrellas con piscina en la provincia de Alicante que aceptan mascotas.
df_comp = df[(df["Stars"] == 3.0) & (df["Postal_Code"].str[0:2] == "03") & (df["Swimming pool"] == 1)
            & (df["Pets allowed"] == 1)]
df_comp

Definimos nuestros precios para los próximos 14 días

In [ ]:
dates = df.Date.unique()
# Próximos días como parámetro
prox_dias = 14
# Nuestros precios para los próximos días (aleatorios)
our_rates = [random.randint(48,69) for j in range(0,prox_dias)]
# Transformamos en dataframe
our_df = pd.DataFrame(zip(dates, our_rates), columns=["Fecha","Rate"])

Comenzamos el análisis

In [ ]:
# Representación
x_values1 = df_comp["Date"].unique()
y_values1 = df_comp.groupby(["Date"])["Price"].mean()
x_values2 = df["Date"].unique()
y_values2 = df.groupby(["Date"])["Price"].mean()
x_values3 = df["Date"].unique()
y_values3 = our_df["Rate"]

plt.figure()    #Figura. 
plt.bar(x_values1, y_values1, color = "red")          #El gráfico de la competencia
plt.bar(x_values2, y_values2, color = "blue", fill = False)          #El gráfico global
plt.plot(x_values3, y_values3, color = "green")          #El gráfico nuestro
plt.title('Precios medios de la competencia los próximos días')      #El título
ax = plt.subplot()                   #Axis
ax.set_xticks(x_values3)             #Eje x
ax.set_xticklabels(x_values3, rotation = 45)        #Etiquetas del eje x
ax.set_xlabel('Fecha')  #Nombre del eje x
ax.set_ylabel('Precio medio')  #Nombre del eje y
plt.show()

In [ ]:
# Generamos una tabla report
df_report = pd.DataFrame(zip(x_values1, our_rates, y_values1, y_values2, y_values1 - our_rates),
            columns=["Fecha", "NuestroPrecio", "Media Competencia", "Media General", "Diferencia"])
df_report

In [ ]:
# Generamos unas alertas que nos ayuden a rectificar los precios y adaptarlos a la competencia
# Las alertas se activarán si nuestro precio está por debajo del 70% o por encima del 10% de la competencia.

def alertas(df_report):
    """Genera a partir de un dataframe reporte, una serie de alertas
        para corregir los precios"""
    for index, row in df_report.iterrows():
        if row["NuestroPrecio"] < 0.7*row["Media Competencia"]:
            print("El día {} es necesario subir el precio".format(row["Fecha"]))
        else:
            if row["NuestroPrecio"] > 1.1*row["Media Competencia"]:
                print("El día {} es necesario bajar el precio".format(row["Fecha"]))
alertas(df_report)   


## La variable 'Comments' y su posible interpretación.

La variable 'Comments' contiene el número de comentarios que cosechan los hoteles en el portal booking.com.
Podemos interpretar el número de comentarios que recibe un hotel dado, como su nivel de popularidad entre los usuarios del portal. A mayor número de comentarios - mayor popularidad del hotel. 

Cada comentario lo podemos considerar como una opinión del cliente que haya adquirido su estancia en este hotel y, pues decide dejar un comentario. Además, realmente nos da igual si el comentario es negativo o positivo, ya que tanto si el cliente se quedó satisfecho como no, el cliente pagó su estancia. Esto implica que, de manera muy generalizada, podemos considerar que, por ejemplo, si un hotel dado recibe 1000 comentarios y su precio de estancia por una noche es de 80 euros, muy hipotéticamente podríamos considerar el siguiente flujo financiero: 1000 comentarios x 80 euros = 80000 euros. 

Con lo cual, a mayor número de comentarios -> mayor popularidad entre los clientes -> mayores flujos entrantes para el hotel. Desde el punto de vista del negocio, nos interesaría estudiar esta variable, su relación con otras variables, en general, nos interesa saber qué cualidades poseen los hoteles que cosechan mayores números de comentarios y gozan de mayor popularidad.

En esta práctica, vamos a trabajar sólamente con las variables que tenemos en el dataset, más que nada por el tiempo que supone el scraping y por tener compasión y piedad con el portal booking.com, pero en teoría se podría recoger muchas más variables y hacer un análisis más completo y exhaustivo. Sin embargo, para demostrar la idea nos sobran las variables que ya tenemos.

In [ ]:
# Filtramos el dataset de manera que sólo nos aparezcan hoteles 1 vez
df_1 = df.drop_duplicates(subset = ["Name"])
df_1.reset_index(drop=True, inplace=True)
df_1.head()

Para poder trabajar con la variable 'Comments', tenemos que pasar los valores al formato adecuado, ya que en el portal el número de comentarios contiene el separador de miles, lo que hace que, por ejemplo, 750 comentarios se interpretan como 750.00, lo que es correcto, mientras que los 1986 comentarios, que en el portal al contener el punto separador de miles(1.986) se interpretan tal cual, 1.986(1 comentario con 986 milésimas), lo que es incorrecto. Con lo cual, tenemos que pasar los valores mayores de 1000 comentarios al formato adecuado.

In [ ]:
# Creamos un vector con valores false/true, de manera que aquellos 
# valores que tienen parte decimal sean 'true', para luego simplemente
# multiplicarlos por 1000, y los valores que no tienen parte 
# decimal(valores inferiores a 1000 comentarios en booking.com) sean 'false'
vec=[]
for i in range(0,len(df_1.Comments)):
    vec.append(int(df_1.Comments.astype(str).str.split('.')[i][1])>0)

In [ ]:
# Multiplicamos valores con parte decimal por 1000 y dejamos los demás
# como están
df_1.loc[:,'Comments']=np.where(vec, df_1.Comments*1000, df_1.Comments)

In [ ]:
df_1.head()

 Vemos que valores superiores a 1000 han tomado el formato adecuado, con lo cual ya podemos proceder con el análisis.

Empezamos con simplemente ver las distribuciones.


In [ ]:
plt.hist(df_1['Comments'], color = 'blue', edgecolor = 'black', fill =True)
plt.show()

In [ ]:
# Hallamos el mínimo
df_1.Comments.min()

In [ ]:
# Hallamos el máximo
df_1.Comments.max()

In [ ]:
# Hallamos la media
df_1.Comments.mean()

Vemos que hay más comentarios hasta 1000, mientras que el mínimo es sólo de 1 comentario y el máximo son 9268 comentarios. La media nos servirá de umbral, el número de comentarios inferior a la media los vamos a considerar como pocos(low), y los que superan a la media, los consideraremos como muchos(high). Vamos a crear una variable binaria, 'Comments_hl'(de Comments_high/low), que contendrá 1 para los comentarios que superan a la media(high) y 0 para comentarios inferiores a la media(low). Está variable la vamos a necesitar para generar las reglas de asociación.

In [ ]:
# Creamos una variable binaria, 1 para los comentarios que superan a la
# media, y 0 para los que no
df_1['Comments_hl'] = np.where(df_1.Comments>df_1.Comments.mean(), 1,0)

In [ ]:
df_1.Comments_hl.isna().any()

Representamos las frecuencias de la variable 'Stars'.

In [ ]:
# Representamos histograma de la variable 'Stars'
plt.hist(df_1['Stars'], bins=np.arange(7)-0.5, color = 'blue', edgecolor = 'black')
plt.xlabel('Stars')
plt.show()

Vemos que hay más hoteles de 4 estrellas y muy pocos(que no llegan a los 100) de 1 y de 5 estrellas.

Ahora sería interesante representar el número de estrellas en función de comentarios recebidos.


In [ ]:
# Representamos las estrellas en función de comentarios
df_1.pivot(columns='Comments_hl', values='Stars').plot.hist(bins=np.arange(7)-0.5, edgecolor='black')
plt.xlabel('Stars')
plt.show()

Está claro que los hoteles de 4 estrellas cosechan el mayor número de comentarios.

La siguiente variable que vamos a ver es el precio.


In [ ]:
# Representamos las frecuencias de los precios
plt.hist(df_1['Price'], color = 'blue', edgecolor = 'black')
plt.xlabel('Price')
plt.show()

Vemos que prevalecen los precios hasta 100 euros. Vamos a aumentar un poco el gráfico exluyendo los precios inferiores a 300 euros.

In [ ]:
# Excluímos los precios inferiores a 300 euros
plt.hist(df_1.Price[df_1.Price>300], color = 'blue', edgecolor = 'black')
plt.xlabel('Price')
plt.show()

Nos convendría discretizar la variable 'Price' partiéndola en intervalos.

In [ ]:
# Partimos en intervalos la variable Price
df_1['Price_cat'] = pd.cut(df_1.Price, [1,100,200,300,400,500,600,700,800,900,1000])

In [ ]:
# Representamos los intervalos del precio
df_1.Price_cat.value_counts().plot.bar(color = 'blue', edgecolor = 'black')
plt.xlabel('Price intervals')
plt.show()

Hay más precios hasta 100 euros. Vamos a representar los precios en función de los comentarios.



In [ ]:
# Representamos los precios en función de los comentarios
df_1.pivot(columns='Comments_hl', values='Price').plot.hist(edgecolor='black')
plt.xlabel('Price')
plt.show()

In [ ]:
# Representamos los precios en función de comentarios excluyendo
# los precios inferiores a 200 euros
df_1[df_1.Price>200].pivot(columns='Comments_hl', values='Price').plot.hist(edgecolor='black')
plt.xlabel('Price')
plt.show()

In [ ]:
# Calculamos cuántos comentarios superiores a la media hay
# en cado intervalos del precios
df_1.Price_cat[df_1.Comments_hl==1].value_counts()

In [ ]:
# Calculamos el valor relativo de los comentarios superiores a la
# media en cada intervalo del precio
for i,n in enumerate(df_1.Price_cat.value_counts()):
    #print(n)
    print(df_1.Price_cat[df_1.Comments_hl==1].value_counts().index[i],list(df_1.Price_cat[df_1.Comments_hl==1].value_counts())[i]/n)
    


Vemos que el intervalo (100, 200] contiene el mayor número de comentarios superiores a la media.

Vamos a representar las variables 'Comments' y 'Price' una contra otra para ver si existe alguna relación de dependencia.


In [ ]:
# Representamos los somentarios contra los precios
plt.scatter(df_1.Comments, df_1.Price)
plt.xlabel('Comments')
plt.ylabel('Price')
plt.show()

No se aprecia relación de dependencia clara entre el precio y el número de comentarios.

## Reglas de asociación.


También podemos aplicar el algoritmo de las reglas de asociación al conjunto de atributos, para descubrir algunos patrones que ocurren en común y que tienen alguna consecuencia. En este caso, nos interesaría descubrir algunas causas(reglas) que hacen que el hotel goce de popularidad(un número de comentarios alto), nos interesa descubrir qué conjunto de atributos(por ejemplo:['Airport Shuttle (free)','BBQ facilities', 'Free WiFi Internet Access Included', '4_Stars']) tienen los hoteles que están altamente populares(tienen un número de comentarios alto) en el portal booking.com. 

Este conjunto de atributos, ['Airport Shuttle (free)','BBQ facilities', 'Free WiFi Internet Access Included', '4_Stars'], se denomina 'antecedente'(X), y el número alto de comentarios(Y), es el consecuente. Con lo cual, la definición formal de una regla sería la implicación X=>Y, que siguiendo con el ejemplo de nuestro caso sería: ['Airport Shuttle (free)','BBQ facilities', 'Free WiFi Internet Access Included', '4_Stars'] => alto nº de comentarios. O sea, nos interesa descubrir conjuntos de antecedentes que implican alto nº de comentarios. 

In [ ]:
# Para poder aplicar el algoritmo de reglas de asociación, tenemos que pasarle
# el dataset en forma binaria(1 o 0). Con lo cual, añadimos a los atributos
# binarios que ya teníamos, algunos más.
df_1['nan'] = np.where(df_1.Stars=='nan', 1,0)
df_1['1_Star'] = np.where(df_1.Stars==1, 1,0)
df_1['2_Stars'] = np.where(df_1.Stars==2, 1,0)
df_1['3_Stars'] = np.where(df_1.Stars==3, 1,0)
df_1['4_Stars'] = np.where(df_1.Stars==4, 1,0)
df_1['5_Stars'] = np.where(df_1.Stars==5, 1,0)
df_1['Price_high'] = np.where(df_1.Price>df_1.Price.mean(), 1,0)
df_1['Price_low'] = np.where(df_1.Price<df_1.Price.mean(), 1,0)

In [ ]:
# El proceso de generación de las reglas de asociación consiste de
# 2 etapas: primero se genera un dataset de artículos frecuentes,
# 'frequent_itemsets', el umbral de frecuencia 'min_support' lo
# establecemos nosotros, lo que quiere decir que quremos generar
# un subset, cuyos elementos(subconjuntos de atributos) ocurren
# simultáneamente con una frecuencia mínima que establezcamos,
# por ejemplo, en un 10% de registros(transacciones), y luego
# pasamos el 'frequent_itemsets' al algoritmo 'association_rules'

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


# Generamos el dataset de artículos frecuentes
frequent_itemsets = apriori(df_1.loc[:, ['Airport Shuttle (free)',
       'BBQ facilities', 'Free WiFi Internet Access Included', 'Wireless Lan',
       'Daily maid service', 'Swimming pool', 'Pets allowed', 'Free Parking',
       'Spa & Wellness Centre', 'Fitness Room', 'Private Beach Area',
       'Heating', 'Family Rooms', 'Coffee/Tea maker', 'Restaurant',
       'Beach front', 'Garden', 'Bar', 'Terrace',
       'Rooms/Facilities for Disabled', 'Elevator', 'Non Smoking Rooms',
       '24 hour Front Desk', 'Room-service', 'Airport Shuttle',
       'Parking (fee required)', 'nan', '1_Star', '2_Stars', '3_Stars', 
        '4_Stars', '5_Stars','Price_high','Price_low','Comments_hl']], min_support=0.1, use_colnames=True)

In [ ]:
# Generamos las reglas de asociación, usando el soporte('support') como métrica
# con un umbral mínimo del 10%. Lo que quiere decir que queremos generar las
# reglas que aparecen como mínimo en un 10% de transacciones.
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.1)
rules.head()

In [ ]:
# Ahora, filtramos las reglas obtenidas de manera que sólo queremos
# aquellas reglas, cuyo consecuente es 'Comments_hl'(el nº de comentarios alto)
# y cuya confianza es mayor del 50%
lft = rules[(rules.consequents=={'Comments_hl'})&
            (rules['confidence']>0.5)].sort_values('confidence', ascending=False)
lft.head(10)

In [ ]:
# Mostramos las 7 primeras reglas con su correspondiente soporte,
# confianza y lift
for index, row in lft.iloc[0:7,[0,4,5,6]].iterrows():
    print(str(row.antecedents)+' support:'+str(round(row.support,2))+
          '; confidence:'+str(round(row.confidence,2))+'; lift: '+str(round(row.lift,2)))
    print()

La interpretación de las reglas de asociación generadas podría ser la siguiente.

La primera regla nos dice que en el dataset hay un 10% de transacciones(registros), support:0.1, con los atributos ['Swimming pool'(piscina), 'Parking(fee required)' y Price_low(precio por debajo de la media)] como antecedente, y de estos 10%, hay un 69% que tienen 'Comments_hl'(el nº alto de comentarios) como consecuente. Es decir, por ejemplo de los 100 transacciones(registros), los 10 tienen este conjunto de atributos, y de estos 10, los casi 7 implican un nº de comentarios alto. Con lo cual, podríamos decir que si un hotel posee estos atributos, con una probabilidad de casi 70%, este hotel tendrá un nº de comentarios elevado.

El indicador 'lift' nos indica la tasa de soporte observada sobre la esperada si X e Y fueran independientes. Un lift < 1 quiere decir que la tasa de soporte observada aparece menos veces que la esperada, mientras que un lift > 1 nos dice que la tasa de soporte observada aparece más veces de la esperada. En nuestro caso, lift(X=>Y)=1.94 quiere decir que es más probable que los sucesos X e Y ocurran juntos, que por separado. En otras palabras, un lift > 1 nos dice que es más probable que el conjunto de atributos ['Swimming pool'(piscina), 'Parking(fee required)' y Price_low(precio por debajo de la media)] implique un nº de comentarios alto, que no. Con lo cual, un lift=1.94 indica que esta regla es bastante fuerte.

La segunda regla tiene los siguientes atributos como antecedente:['Swimming pool', '4_Stars', 'Bar', 'Parking (fee required)']. Este conjunto aparece en un 11% de transacciones(registros), support=0.11. De este 11% de transacciones, en un 69% implica el nº de comentarios alto, confidence=0.69. Un lift=1.93 nos indica que esta regla también es bastante fuerte. Con lo cual, podríamos afirmar que si un hotel tiene piscina, bar, parking y es de 4 estrellas, con una probabilidad de casi 70% este hotel cosecharía un nº de comentarios alto.

Todas las demás reglas se interpretan de la misma manera, sólo que cambian los conjuntos de atributos, y los indicadores 'support', 'confidence' y 'lift' varían ligeramente.
